In [1]:
import pandas as pd

# Merging of the 4 leaks folders

Paradise papers are not included for the moment.
This short code makes correspond the new version of the dataset with the former one.

In [7]:
leaks= ['bahamas_leaks', 'offshore_leaks', 'panama_papers'] #paradise_papers
month_code = '2017-11-17'

edges_df = pd.DataFrame([])
addresses_df = pd.DataFrame([])
entities_df = pd.DataFrame([])
intermediaries_df = pd.DataFrame([])
officers_df = pd.DataFrame([])


for leak in leaks:
    edges_df = edges_df.append(pd.read_csv('./data/data_csv/csv_' + leak + '.' +
                                           month_code + '/' + leak + '.edges.csv', dtype=object))
    
    addresses_df = addresses_df.append(pd.read_csv('./data/data_csv/csv_' + leak + '.' + 
                                               month_code + '/' + leak + '.nodes.address.csv', dtype=object))
    
    entities_df = entities_df.append(pd.read_csv('./data/data_csv/csv_' + leak + '.' + 
                                           month_code + '/' + leak + '.nodes.entity.csv', dtype=object))
    
    intermediaries_df = intermediaries_df.append(pd.read_csv('./data/data_csv/csv_' + leak + '.' + 
                                           month_code + '/' + leak + '.nodes.intermediary.csv', dtype=object))
    
    officers_df = officers_df.append(pd.read_csv('./data/data_csv/csv_' + leak + '.' + 
                                           month_code + '/' + leak + '.nodes.officer.csv', dtype=object))
    
    
edges_df.index = range(len(edges_df))
edges_df.to_csv('./data/data_csv/all_edges.csv')
addresses_df.index = range(len(addresses_df))
addresses_df.columns = addresses_df.columns.str.replace('n\.','')
addresses_df.to_csv('./data/data_csv/Addresses.csv')
entities_df.index = range(len(entities_df))
entities_df.columns = entities_df.columns.str.replace('n\.','')
entities_df.to_csv('./data/data_csv/Entities.csv')
intermediaries_df.index = range(len(intermediaries_df))
intermediaries_df.columns = intermediaries_df.columns.str.replace('n\.','')
intermediaries_df.to_csv('./data/data_csv/Intermediaries.csv')
officers_df.index = range(len(officers_df))
officers_df.columns = officers_df.columns.str.replace('n\.','')
officers_df.to_csv('./data/data_csv/Officers.csv')

# Reading and cleaning of the dataset

Reading of the different parts of the dataset. There are 4 different files that are converted into pandas DataFrames:
- `Entities.csv`, `Officers.csv`, `Intermediaries.csv` are dedicated to the three types of actors encountered in the database. Entities refer to asset providers and officers to financial actors (company, private client, ...). Intermediaries refer to actors putting clients and financial service providers in contact.
- `Addresses.csv` describe all the addresses contained in the database those addresses are linked to officers.
- `all_edges.csv` describe the relationships between the items of the database described before, that are entities, officers, intermediaries and addresses. Four different kinds of relationships are described in this dataset: 'registered address', 'shareholder of', 'beneficiary of' and 'intermediary of'.



In [8]:
entities = pd.read_csv("./data/data_csv/Entities.csv", dtype = 'object')
intermediaries = pd.read_csv("./data/data_csv/Intermediaries.csv", dtype = 'object')
officers = pd.read_csv("./data/data_csv/Officers.csv", dtype = 'object')
addresses = pd.read_csv("./data/data_csv/Addresses.csv", dtype = 'object')
all_edges = pd.read_csv("./data/data_csv/all_edges.csv", dtype = 'object')

### Dataset description

Now we print the DataFrames' columns and size in order to have a rough idea of their content.

In [9]:
print('entities:')
print('\tshape:', entities.shape)
print('\tcolumns:', entities.columns)
print()

print('intermediaries:')
print('\tshape:', intermediaries.shape)
print('\tcolumns:', intermediaries.columns)
print()

print('officers:')
print('\tshape:', officers.shape)
print('\tcolumns:', officers.columns)
print()

print('addresses:')
print('\tshape:', addresses.shape)
print('\tcolumns:', addresses.columns)
print()

print('all_edges:')
print('\tshape:', all_edges.shape)
print('\tcolumns:', all_edges.columns)

entities:
	shape: (495038, 19)
	columns: Index(['Unnamed: 0', 'labels(n)', 'valid_until', 'country_codes', 'countries',
       'node_id', 'sourceID', 'address', 'name', 'jurisdiction_description',
       'service_provider', 'jurisdiction', 'closed_date', 'incorporation_date',
       'ibcRUC', 'type', 'status', 'company_type', 'note'],
      dtype='object')

intermediaries:
	shape: (24177, 19)
	columns: Index(['Unnamed: 0', 'labels(n)', 'valid_until', 'country_codes', 'countries',
       'node_id', 'sourceID', 'address', 'name', 'jurisdiction_description',
       'service_provider', 'jurisdiction', 'closed_date', 'incorporation_date',
       'ibcRUC', 'type', 'status', 'company_type', 'note'],
      dtype='object')

officers:
	shape: (370854, 19)
	columns: Index(['Unnamed: 0', 'labels(n)', 'valid_until', 'country_codes', 'countries',
       'node_id', 'sourceID', 'address', 'name', 'jurisdiction_description',
       'service_provider', 'jurisdiction', 'closed_date', 'incorporation_date'

### Cleaning up

The `entities`, `intermediaries` and `officers`' rows whose columns `name` and `countries` contains a NaN value are dropped because these rows will not be exploitable for the analysis. We also drop `addresses`' rows whose `address` column has a NaN value.

In [10]:
original_num_row = entities.shape[0]
entities = entities[entities['name'].notnull()]
entities = entities[entities['countries'].notnull()]
entities = entities[entities['country_codes'] != 'XXX']
entities[entities['country_codes'] == 'REU'] = 'FRA'
entities[entities['country_codes'] == 'MTQ'] = 'FRA'
final_num_row = entities.shape[0]
print(original_num_row - final_num_row, 'rows dropped in entities')

original_num_row = intermediaries.shape[0]
intermediaries = intermediaries[intermediaries['name'].notnull()]
intermediaries = intermediaries[intermediaries['countries'].notnull()]
intermediaries = intermediaries[intermediaries['country_codes'] != 'XXX']
intermediaries[intermediaries['country_codes'] == 'REU'] = 'FRA'
intermediaries[intermediaries['country_codes'] == 'MTQ'] = 'FRA'
final_num_row = intermediaries.shape[0]
print(original_num_row - final_num_row, 'rows dropped in intermediaries')

original_num_row = officers.shape[0]
officers = officers[officers['name'].notnull()]
officers = officers[officers['countries'].notnull()]
officers = officers[officers['country_codes'] != 'XXX']
officers[officers['country_codes'] == 'REU'] = 'FRA'
officers[officers['country_codes'] == 'MTQ'] = 'FRA'
final_num_row = officers.shape[0]
print(original_num_row - final_num_row, 'rows dropped in officers')

original_num_row = addresses.shape[0]
addresses = addresses[addresses['address'].notnull()]
addresses = addresses[addresses['countries'].notnull()]
addresses = addresses[addresses['country_codes'] != 'XXX']
addresses[addresses['country_codes'] == 'REU'] = 'FRA'
addresses[addresses['country_codes'] == 'MTQ'] = 'FRA'
final_num_row = addresses.shape[0]
print(original_num_row - final_num_row, 'rows dropped in addresses')

187862 rows dropped in entities
1909 rows dropped in intermediaries
158254 rows dropped in officers
1247 rows dropped in addresses


### Writing clean datasets in new files

In order to not have to carry out the preprocessing again we write the the DataFrames in new files.

In [11]:
entities.to_csv('./data/data_clean_csv/entities_clean.csv', index = False)
intermediaries.to_csv('./data/data_clean_csv/intermediaries_clean.csv', index = False)
officers.to_csv('./data/data_clean_csv/officers_clean.csv', index = False)
addresses.to_csv('./data/data_clean_csv/addresses_clean.csv', index = False)
all_edges.to_csv('./data/data_clean_csv/all_edges_clean.csv', index = False)